In [28]:
import requests
import numpy as np
import pandas as pd
import sys

In [ ]:
# Get race data starting from 1995

import requests
import pandas as pd

def get_race_data(start_year=1995):
    base_url = "http://ergast.com/api/f1"
    current_year = 2024  # Update this to the current year
    all_races = []

    for year in range(start_year, current_year):
        year_url = f"{base_url}/{year}.json"
        response = requests.get(year_url)
        if response.status_code == 200:
            year_data = response.json()
            try:
                races = year_data['MRData']['RaceTable']['Races']
                for race in races:
                    race_info = {
                        'season': race['season'],
                        'round': race['round'],
                        'raceName': race['raceName'],
                        'date': race['date'],
                        'time': race.get('time', 'N/A'),  # 'time' might not be available for all races
                        'circuitId': race['Circuit']['circuitId'], # TODO: Sarah added this line - wondering if it's useful to grab the id too 
                        'circuit': race['Circuit']['circuitName'],
                        'location': race['Circuit']['Location']['locality'],
                        'country': race['Circuit']['Location']['country'],
                        'long': race['Circuit']['Location']['long'], #TODO: Sarah added - was thinking it could be useful if we want to tie in wheather/terrain data or for vis
                        'lat':  race['Circuit']['Location']['lat'],# TODO: same comment as above
                    }
                    all_races.append(race_info)
            except KeyError:
                print(f"Data format error for year {year}")
        else:
            print(f"Failed to fetch data for year {year}")

    return pd.DataFrame(all_races)

# Fetch the data and create a DataFrame
race_data_df = get_race_data()
race_data_df.to_csv("../data/raw/Race_Information_1995_2023.csv")

In [34]:
race_dfs = []

for index, row in race_data_df.iterrows():
    season = row['season']
    round_num = row['round']
    results_url = f"http://ergast.com/api/f1/{season}/{round_num}/driverStandings.json"
    sys.stdout.write(f"\rFetching results for Season: {season}, Round: {round_num}. Url: {results_url}\n")
    sys.stdout.flush()
    response = requests.get(results_url)

    if response.status_code == 200:
        results_data = response.json()
        try:
            results = results_data['MRData']['StandingsTable']['StandingsLists']['DriverStandings']
            results_df = pd.DataFrame(results)
            results_df['season'] = season
            results_df['round'] = round_num
            race_dfs.append(results_df)
        except KeyError:
                print(f"Data format error for season {season}, round {round_num}")
    else:
        print(f"Failed to fetch data for season {season}, round {round_num}")

Fetching results for Season: 1995, Round: 1. Url: http://ergast.com/api/f1/1995/1/driverStandings.json


TypeError: list indices must be integers or slices, not str

In [ ]:
base_url = "http://ergast.com/api/f1"
current_year = 2024
start_year = 1995

for year in range(start_year,current_year):
    year_url = f"{base_url}/{year}.json"
    response = requests.get(year_url)
    

In [ ]:
response.json()

In [ ]:
requests.get("http://ergast.com/api/f1/driverStandings/1").json()